### RNN

RNN的出现解决了NPLM的几个问题，比如窗口固定和不能很好的处理长距离依赖关系。     

RNN可以看作是一个具有"记忆"的神经网络，其基本原理是通过循环来传递隐藏状态信息，从而实现对序列数据的建模。       

RNN的基本组成为输入层、隐藏层和输出层。在每一个时间步，RNN会通过当前输入和上一个时间步的隐藏状态来计算更新这个时间步的隐藏状态，这个隐藏状态同时会用来计算当前的输出和更新下一个时间步的隐藏状态，同时RNN具有参数共享的特征，在不同的时间步，RNN使用相同的权重矩阵和偏置。RNN基本单元和按照时间展开的结构如图：

<img src="./images/RNN.png" alt="img" style="zoom: 50%;" />

RNN采用BPTT(基于时间的反向传播)算法进行训练, 也就是将梯度沿着时间步反向传播，从输出层一直传播到到输入层。因为RNN层使用相同的权重，因此最终权重梯度是各个RNN层的权重梯度之和。

需要注意的是RNN也有有些局限性，比如在训练过程中会遇到梯度消失和梯度爆炸的问题，很难学习长距离依赖关系，因此也就出现了基于门控机制的LSTM和GRU

### RNN的具体实现
整体与NPLM非常相似，只是模型结构有差别

> 1. 构建实验语料

In [3]:
sentences = ["我 喜欢 玩具", "我 爱 爸爸", "我 讨厌 挨打"] 
# 创建词汇表
word_list = list(set(" ".join(sentences).split()))
# index -> word
index_to_word = { index: word for index,word in enumerate(word_list)}
word_to_index = { word: index for index,word in enumerate(word_list)}
print("词汇表长度",len(word_list))
print(word_to_index)
voc_size = len(word_list)

词汇表长度 7
{'讨厌': 0, '我': 1, '爱': 2, '挨打': 3, '爸爸': 4, '玩具': 5, '喜欢': 6}


> 2. 生成训练数据

In [22]:
# 从语料库中生成批处理数据，一批一批的喂入网络进行训练
import torch
import random
batch_size = 2
def make_batch(batch_size=2):
    input_batch = []
    target_batch = []
    sample_sentences = random.sample(sentences,batch_size) # 通过random随机选择数据
    # print(sample_sentences)
    for sentence in sample_sentences:
        # 通过空格分隔词汇
        split_sentences = sentence.split()
        input = [word_to_index[ele] for ele in split_sentences[:-1]] # 将前n-1个词汇的索引作为输入
        target = word_to_index[split_sentences[-1]] # 目标词汇为最后一个词
        input_batch.append(input)
        target_batch.append(target)
    input_batch = torch.LongTensor(input_batch)
    target_batch = torch.LongTensor(target_batch)
    return input_batch, target_batch
input_batch, targe_batch = make_batch(batch_size)
print("打印训练数据", input_batch)

打印训练数据 tensor([[1, 2],
        [1, 0]])


> 3. 构建RNN模型(LSTM)

In [23]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.C = nn.Embedding(voc_size,embedding_size)
        # 用lstm层替代第一个线性层, batch_first=True表示输入的数据的第一个维度为batch_size
        self.lstm = nn.LSTM(embedding_size, n_hidden, batch_first=True)
        self.linear = nn.Linear(n_hidden, voc_size)
    
    def forward(self, X):
        # 输入X的形状为[batch_size, n_step]
        X = self.C(X) # 通过emdedding后变为:[batch_size, n_step, embedding_size]
        lstm_out,_ = self.lstm(X) # lstm_out的形状为[batch_size, n_step, n_hidden]
        output = self.linear(lstm_out[:,-1,:]) # 输出的形状为[batch_size, voc_size]
        return output
        # output, (hn, cn) = rnn(input, (h0, c0))

In [24]:
n_step = 2 # 时间步数，表示每个输入序列的长度，也就是上下文长度 
n_hidden = 2 # 隐藏层大小
embedding_size = 2 # 词嵌入大小
model = RNN() # 创建神经概率语言模型实例
print(' RNN 模型结构：', model) # 打印模型的结构

 RNN 模型结构： RNN(
  (C): Embedding(7, 2)
  (lstm): LSTM(2, 2, batch_first=True)
  (linear): Linear(in_features=2, out_features=7, bias=True)
)


> 4. 模型训练

In [26]:
import torch.optim as optim # 导入优化器模块
criterion = nn.CrossEntropyLoss() # 定义损失函数为交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.1) # 定义优化器为 Adam，学习率为 0.1
# 训练模型
for epoch in range(5000): # 设置训练迭代次数
   optimizer.zero_grad() # 清除优化器的梯度
   input_batch, target_batch = make_batch() # 创建输入和目标批处理数据
   output = model(input_batch) # 将输入数据传入模型，得到输出结果
   loss = criterion(output, target_batch) # 计算损失值
   if (epoch + 1) % 1000 == 0: # 每 1000 次迭代，打印损失值
     print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
   loss.backward() # 反向传播计算梯度
   optimizer.step() # 更新模型参数

Epoch: 1000 cost = 0.000000
Epoch: 2000 cost = 0.000000
Epoch: 3000 cost = 0.000000
Epoch: 4000 cost = 0.000000
Epoch: 5000 cost = 0.000000


> 5. 预测数据

In [34]:
# 进行预测
input_strs = [['我', '讨厌'], ['我', '喜欢']]  # 需要预测的输入序列
# 将输入序列转换为对应的索引
input_indices = [[word_to_index[word] for word in seq] for seq in input_strs]
# 将输入序列的索引转换为张量
input_batch = torch.LongTensor(input_indices) 
# 对输入序列进行预测，取输出中概率最大的类别
predict = model(input_batch).data.max(1)[1]  
# 将预测结果的索引转换为对应的词
print(predict)
print(predict.squeeze())
predict_strs = [index_to_word[n.item()] for n in predict.squeeze()]  
for input_seq, pred in zip(input_strs, predict_strs):
   print(input_seq, '->', pred)  # 打印输入序列和预测结果

tensor([3, 5])
tensor([3, 5])
['我', '讨厌'] -> 挨打
['我', '喜欢'] -> 玩具


### RNN基本实例的介绍
来源于源码处的demo,介绍下每个参数及输出的意义

In [36]:
rnn = nn.LSTM(10, 20, 2) # batch_first = false, 因此输入数据的第一个维度为序列的长度即n_step
"""
10 : 输入特征的维度
20 : 隐藏状态的维度大小, 也就是LSTM单元中隐藏状态大小为20
2 : 双层LSTM
"""
input = torch.randn(5, 3, 10)
"""
5 : 输入序列的长度, 也就是时间步数为5
3 : 批次大小, 也就是每个时间步同时处理的序列个数为3
10 : 输入特征的维度大小隐
"""
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
"""
初始化模型的隐藏状态和细胞状态
2 : lstm的层数
3 : 批次大小
20 : 隐藏状态的维度
"""
output, (hn,cn) = rnn(input, (h0, c0))
print(output.shape) # 模型最终输出, 包含了在每个时间步的每个batch的输出(隐藏状态(ht))
print(hn.shape) 
print(cn.shape)
# hn和cn表示每个batch在最后一个时间步的隐藏状态和细胞状态

torch.Size([5, 3, 20])
torch.Size([2, 3, 20])
torch.Size([2, 3, 20])


### RNN存在的问题

- 顺序计算: 每个时间步的数据是顺序计算的, 也就是当前时间步必须在上一个时间步计算完以后才能计算, 限制了网络的并行计算能力, 降低计算效率和速度
- 长距离依赖问题: 尽管GRU和LSTM通过门控机制有效了改善了这个问题，但是在序列非常长时，仍然无法完成捕捉到序列中的长距离依赖问题
- 扩展性: 在涉及到大规模数据时, 模型的可扩展性较弱。随着序列长度的增加, 计算复杂性会变高, 导致训练时间过长
- 在NLP落地方面: 模型表达能力不足, 缺乏大规模数据, 优化算法发展不足
  
以上问题最终会在注意力机制和transformer的出现之后尘埃落定!!